In [36]:
from math import log
from sklearn import datasets
import numpy as np
import pandas as pd

def entropy(column):
    probabilities=[]
    for x in column.value_counts():
        probabilities.append(x/sum(col1.value_counts()))
    return -sum([p*log(p,2) for p in probabilities])

In [143]:
iris=datasets.load_iris()
iris_data=iris['data']
iris_target=iris['target']
data=[]
for index,row in enumerate(iris_data):
    new_row=np.append(row,[iris_target[index]])
    data.append(new_row.tolist())
    
# data #list of lists
columns=iris.feature_names
columns.append('species')
df=pd.DataFrame(data,columns=columns)
df.describe()
# df.dtypes

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [195]:
def importance(attributes,examples):
    label_column=examples.iloc[:,-1:]
    current_entropy=entropy(label_column)
    max_gain=0.0
    important_attribute=attributes[0]
    for a in attributes:
        col=examples[a]
        remainder=0.0
        for v in col.unique():
            subset=examples[examples[a]==v]
            subset_entropy=entropy(subset.iloc[:,-1:])
            subset_probability=len(subset)/len(examples)
            remainder=remainder+(subset_entropy*subset_probability)
        gain=current_entropy-remainder
        if gain>max_gain:
            max_gain=gain
            important_attribute=a
    return important_attribute
A=importance(df.columns.drop('species'),df)
A

'petal length (cm)'

In [269]:
class Node:
    def __init__(self):
        self.children = []
        self.branch_value = None
        self.attribute = None
        self.leaf=None
        
        
    def __str__(self):
        return """
            leaf:{0},
            attribute:{1},
            branch_value:{2},
            children:{3}
        """.format(self.leaf,self.attribute,self.branch_value,self.children)

In [270]:
def learn_decision_tree(examples,attributes,parent_examples,branch_value=None,parent=None):
    """
        Expects:
            - examples: The branch examples - Pandas Dataframe including the label. Label should be the last column
            - attributes: List/Pandas Index of the attributes excluding the label
            - parent_examples: The Parent Node examples - Pandas Dataframe including the label. Label should be the last column
    """
    if examples.empty:
#         print("Examples exhausted")
        node=Node()
        node.branch_value=branch_value
        node.leaf=max(parent_examples.iloc[:,-1:]).index
        return node
    elif len(examples.iloc[:,-1].unique())==0:
#         print("Reached Leaf with Uniform Class")
        node=Node()
        node.branch_value=branch_value
        node.leaf=examples.iloc[:,-1:][0]
        return node
    elif len(attributes)==0:
#         print("Attributes exhausted")
        node=Node()
        node.branch_value=branch_value
        node.leaf=examples.iloc[:,-1:].value_counts().index
        return node
    else:
#         print("Got Attributes :",str(attributes))
#         print("Got examples :",str(examples))
        A=importance(attributes,examples)
        
#         print("Got ATTRIBUTE :"+str(A))
        node=Node()
        node.attribute=A
        node.branch_value=branch_value
#         print("\n --- \n")
        
        for v in examples[A].unique():
            subset=examples[examples[A]==v]
            new_attributes_set=attributes.drop(A)
            subtree=learn_decision_tree(subset,new_attributes_set,examples,branch_value=v)
            node.children.append(subtree)
        
        return node
 
        
def predict_from_tree(instance,tree):
    pass
attributes=df.columns.drop('species')
tree=learn_decision_tree(df,attributes,df)

In [271]:
print(tree)
for c in tree.children:
    print(c)


            leaf:None,
            attribute:petal length (cm),
            branch_value:None,
            children:[<__main__.Node object at 0x7f7ea10605b0>, <__main__.Node object at 0x7f7ea1138790>, <__main__.Node object at 0x7f7ea11383d0>, <__main__.Node object at 0x7f7ea1012a00>, <__main__.Node object at 0x7f7ea12bcaf0>, <__main__.Node object at 0x7f7ea11128b0>, <__main__.Node object at 0x7f7ea11a07c0>, <__main__.Node object at 0x7f7ea112f070>, <__main__.Node object at 0x7f7ea12b1940>, <__main__.Node object at 0x7f7ea10b1d30>, <__main__.Node object at 0x7f7ea12b1c40>, <__main__.Node object at 0x7f7ea103e9d0>, <__main__.Node object at 0x7f7ea10f4970>, <__main__.Node object at 0x7f7ea11064c0>, <__main__.Node object at 0x7f7ea1008b20>, <__main__.Node object at 0x7f7ea1071310>, <__main__.Node object at 0x7f7ea1106100>, <__main__.Node object at 0x7f7ea11035e0>, <__main__.Node object at 0x7f7ea1120190>, <__main__.Node object at 0x7f7ea110c6a0>, <__main__.Node object at 0x7f7ea1008100>, 